##ThinkDSP

This notebook contains code solutions to exercises in Chapter 6: Discrete Cosine Transform

Copyright 2015 Allen Downey

License: [Creative Commons Attribution 4.0 International](http://creativecommons.org/licenses/by/4.0/)

In [ ]:
from __future__ import print_function, division

import thinkdsp
import thinkplot

import numpy as np
import scipy.fftpack

import autocorr
import dct

PI2 = 2 * math.pi

%matplotlib inline

**Exercise:** Test the algorithmic complexity of `analyze1`, `analyze2` and `scipy.fftpack.dct`.

In [ ]:
signal = thinkdsp.UncorrelatedGaussianNoise()
noise = signal.make_wave(duration=1.0)

ns = 2 ** np.arange(6, 14)
ns

In [ ]:
def plot_bests(bests):    
    thinkplot.plot(ns, bests)
    thinkplot.config(xscale='log', yscale='log', legend=False)
    
    x = np.log(ns)
    y = np.log(bests)
    t = scipy.stats.linregress(x,y)
    slope = t[0]

    return slope

In [ ]:
results = []
for N in ns:
    ts = (0.5 + np.arange(N)) / N
    freqs = (0.5 + np.arange(N)) / 2
    ys = noise.ys[:N]
    result = %timeit -o dct.analyze1(ys, freqs, ts)
    results.append(result)

bests = [result.best for result in results]

thinkplot.preplot(3)
plot_bests(bests)

In [ ]:
results = []
for N in ns:
    ts = (0.5 + np.arange(N)) / N
    freqs = (0.5 + np.arange(N)) / 2
    ys = noise.ys[:N]
    result = %timeit -o dct.analyze2(ys, freqs, ts)
    results.append(result)

bests2 = [result.best for result in results]
plot_bests(bests2)

In [ ]:
results = []
for N in ns:
    ys = noise.ys[:N]
    result = %timeit -o scipy.fftpack.dct(ys, type=3)
    results.append(result)

bests3 = [result.best for result in results]
plot_bests(bests3)

In [ ]:
thinkplot.preplot(3)
thinkplot.plot(ns, bests, label='analyze1')
thinkplot.plot(ns, bests2, label='analyze2')
thinkplot.plot(ns, bests3, label='fftpack.dct')
thinkplot.config(xscale='log', yscale='log', legend=True, loc='upper left')

5) One of the major applications of the DCT is compression for both sound and images. In its simplest form, DCT-based compression works like this:

1. Break a long signal into segments.
2. Compute the DCT of each segment.
3. Identify frequency components with amplitudes so low they are inaudible, and remove them. Store only the frequencies and amplitudes that remain.
4. To play back the signal, load the frequencies and amplitudes for each segment and apply the inverse DCT.

Implement a version of this algorithm and apply it to a recording of music or speech. How many components can you eliminate before the difference is perceptible?

In [ ]:
def make_dct_spectrogram(wave, seg_length, window_func=np.hamming):
    """Computes the DCT spectrogram of the wave.

    seg_length: number of samples in each segment
    window_func: function used to compute the window

    returns: Spectrogram
    """
    n = len(wave.ys)
    window = window_func(seg_length)

    start, end, step = 0, seg_length, seg_length / 2
    spec_map = {}

    while end < n:
        ys = wave.ys[start:end] * window

        t = (start + end) / 2.0 / wave.framerate
        spec_map[t] = thinkdsp.Wave(ys, wave.framerate).make_dct()

        start += step
        end += step

    return thinkdsp.Spectrogram(spec_map, seg_length, window_func)

In [ ]:
wave = thinkdsp.read_wave('100475__iluppai__saxophone-weep.wav')
#wave = thinkdsp.read_wave('92002__jcveliz__violin-origional.wav')

In [ ]:
segment = wave.segment(start=1.2, duration=0.5)
segment.normalize()
segment.make_audio()

In [ ]:
dct = segment.make_dct()
dct.plot()

In [ ]:
def compress(dct):
    thresh = 1000
    count = 0
    for i, amp in enumerate(dct.amps):
        if abs(amp) < thresh:
            dct.amps[i] = 0
            count += 1
            
    print(count, len(dct.amps))

In [ ]:
spectro = make_dct_spectrogram(wave, seg_length=1024)
for t, dct in sorted(spectro.spec_map.iteritems()):
    compress(dct)

In [ ]:
wave2 = spectro.make_wave()
wave2.make_audio()